# Predicting Home Prices in Banglore (Regression Problem)

## Import the require library

In [3]:
import pandas as pd
import numpy as np

## Load the dataset

In [5]:
df = pd.read_csv("bengaluru_house_prices.csv")
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [6]:
df.shape

(13320, 9)

## Drop unnecessary columns

In [7]:
df.drop(columns=['area_type','availability','society'],inplace=True)

In [8]:
df.shape

(13320, 6)

In [9]:
df.dropna(inplace=True)
df.shape

(12710, 6)

## Clean the data for location column

In [10]:
df.location.value_counts()

location
Whitefield         514
Sarjapur  Road     372
Electronic City    300
Kanakpura Road     261
Thanisandra        231
                  ... 
Gulakamale           1
Duddanahalli         1
Doddanakunte         1
Jogupalya            1
Abshot Layout        1
Name: count, Length: 1265, dtype: int64

In [11]:
df['location'] = df['location'].apply(lambda x: x.strip())

In [12]:
location_stats = df.groupby('location')['location'].agg('count').sort_values(ascending=False)

In [13]:
location_less_than_10 = location_stats[location_stats<=10]
location_less_than_10

location
Basapura                   10
Gunjur Palya               10
Dairy Circle               10
HAL 2nd Stage              10
Kalkere                    10
                           ..
mvj engineering college     1
manyata tech park           1
manyata                     1
kg halli jalhalli west      1
kanakapura road             1
Name: location, Length: 1017, dtype: int64

In [14]:
df['location'] = df['location'].apply(lambda x: 'other' if x in location_less_than_10 else x)

In [15]:
df['location'].value_counts()

location
other               2739
Whitefield           515
Sarjapur  Road       372
Electronic City      302
Kanakpura Road       261
                    ... 
Marsur                11
Karuna Nagar          11
Prithvi Layout        11
Doddaballapur         11
Thyagaraja Nagar      11
Name: count, Length: 238, dtype: int64

In [16]:
df

,location,size,total_sqft,bath,balcony,price
0,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Kothanur,2 BHK,1200,2.0,1.0,51.00
...,...,...,...,...,...,...
13314,Green Glen Layout,3 BHK,1715,3.0,3.0,112.00
13315,Whitefield,5 Bedroom,3453,4.0,0.0,231.00
13317,Raja Rajeshwari Nagar,2 BHK,1141,2.0,1.0,60.00
13318,Padmanabhanagar,4 BHK,4689,4.0,1.0,488.00


## Clean the data for size column

In [17]:
df['size'].unique()

array(['2 BHK', '4 Bedroom', '3 BHK', '3 Bedroom', '1 BHK', '1 RK',
       '4 BHK', '1 Bedroom', '2 Bedroom', '6 Bedroom', '8 Bedroom',
       '7 Bedroom', '5 BHK', '7 BHK', '6 BHK', '5 Bedroom', '11 BHK',
       '9 BHK', '9 Bedroom', '27 BHK', '11 Bedroom', '43 Bedroom',
       '14 BHK', '8 BHK', '12 Bedroom', '10 Bedroom', '13 BHK'],
      dtype=object)

In [18]:
df['bedrooms'] = df['size'].apply(lambda x: int(x.split(' ')[0]))

In [19]:
df.head()

,location,size,total_sqft,bath,balcony,price,bedrooms
0,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07,2
1,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00,4
2,Uttarahalli,3 BHK,1440,2.0,3.0,62.00,3
3,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00,3
4,Kothanur,2 BHK,1200,2.0,1.0,51.00,2


## Clean the data for total_sqft column

In [20]:
df.total_sqft.unique()

array(['1056', '2600', '1440', ..., '1133 - 1384', '774', '4689'],
      shape=(1976,), dtype=object)

In [21]:
def clean_sqft(sqft):
  tokens = sqft.split('-')
  if len(tokens) == 2:
    return (float(tokens[0])+float(tokens[1]))/2
  try:
    return float(sqft)
  except:
    return None

In [22]:
df['total_sqft'] = df['total_sqft'].apply(clean_sqft)

In [23]:
df.total_sqft.unique()

array([1056. , 2600. , 1440. , ..., 1258.5,  774. , 4689. ], shape=(1887,))

In [24]:
df.dropna(inplace=True)

In [25]:
df.describe()

,total_sqft,bath,balcony,price,bedrooms
count,12668.000000,12668.000000,12668.000000,12668.000000,12668.000000
mean,1511.835167,2.616277,1.585649,105.952648,2.736422
std,1162.097276,1.223882,0.816758,131.813137,1.202643
min,5.000000,1.000000,0.000000,8.000000,1.000000
25%,1100.000000,2.000000,1.000000,49.015000,2.000000
50%,1260.000000,2.000000,2.000000,70.000000,3.000000
75%,1640.000000,3.000000,2.000000,115.000000,3.000000
max,52272.000000,40.000000,3.000000,2912.000000,43.000000


In [26]:
df['sqft_per_bed'] = df['total_sqft']/df['bedrooms']

In [27]:
df.head()

,location,size,total_sqft,bath,balcony,price,bedrooms,sqft_per_bed
0,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2,528.0
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4,650.0
2,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3,480.0
3,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3,507.0
4,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2,600.0


In [28]:
df.sqft_per_bed.describe()

count    12668.000000
mean       570.060291
std        380.298999
min          0.714286
25%        473.333333
50%        550.000000
75%        622.500000
max      26136.000000
Name: sqft_per_bed, dtype: float64

In [29]:
df = df[df['sqft_per_bed']>=300]

In [30]:
df.loc[:, 'Price_per_sqft'] = round(df['price'] * 100000 / df['total_sqft'], 2)

C:\Users\Darshan\AppData\Local\Temp\ipykernel_20980\2157589900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Price_per_sqft'] = round(df['price'] * 100000 / df['total_sqft'], 2)


In [31]:
df.Price_per_sqft.describe()

count     12013.000000
mean       6206.082361
std        3985.518849
min         267.830000
25%        4199.360000
50%        5252.530000
75%        6823.530000
max      176470.590000
Name: Price_per_sqft, dtype: float64

In [32]:
df = df[df['Price_per_sqft']>=2000]
df

,location,size,total_sqft,bath,balcony,price,bedrooms,sqft_per_bed,Price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,1.0,39.07,2,528.000000,3699.81
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,3.0,120.00,4,650.000000,4615.38
2,Uttarahalli,3 BHK,1440.0,2.0,3.0,62.00,3,480.000000,4305.56
3,Lingadheeranahalli,3 BHK,1521.0,3.0,1.0,95.00,3,507.000000,6245.89
4,Kothanur,2 BHK,1200.0,2.0,1.0,51.00,2,600.000000,4250.00
...,...,...,...,...,...,...,...,...,...
13314,Green Glen Layout,3 BHK,1715.0,3.0,3.0,112.00,3,571.666667,6530.61
13315,Whitefield,5 Bedroom,3453.0,4.0,0.0,231.00,5,690.600000,6689.83
13317,Raja Rajeshwari Nagar,2 BHK,1141.0,2.0,1.0,60.00,2,570.500000,5258.55
13318,Padmanabhanagar,4 BHK,4689.0,4.0,1.0,488.00,4,1172.250000,10407.34


In [33]:
df.drop(columns=['size', 'sqft_per_bed', 'Price_per_sqft'], axis=1, inplace=True)

In [34]:
df.head()

,location,total_sqft,bath,balcony,price,bedrooms
0,Electronic City Phase II,1056.0,2.0,1.0,39.07,2
1,Chikka Tirupathi,2600.0,5.0,3.0,120.00,4
2,Uttarahalli,1440.0,2.0,3.0,62.00,3
3,Lingadheeranahalli,1521.0,3.0,1.0,95.00,3
4,Kothanur,1200.0,2.0,1.0,51.00,2


## Model Creation

In [36]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [37]:
col_trans = make_column_transformer((OneHotEncoder(sparse_output=False), ['location']), remainder='passthrough')

In [38]:
regression = LinearRegression()
scaler = StandardScaler()

In [39]:
model = make_pipeline(col_trans, scaler, regression)

In [40]:
X = df.drop(columns=['price'])
y = df['price']

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [42]:
X_train.head(1)

,location,total_sqft,bath,balcony,bedrooms
317,Whitefield,1280.0,2.0,2.0,2


In [43]:
model.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('standardscaler', ...), ...]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('onehotencoder', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the dif

In [44]:
y_pred = model.predict(X_test)

In [45]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
score = r2_score(y_test, y_pred)
print(score)

mse = mean_squared_error(y_test, y_pred)
print(mse)

mae = mean_absolute_error(y_test, y_pred)
print(mae)

0.6742344251447545
5254.853841439894
35.70803266321278


In [46]:
example = pd.DataFrame([{'location': 'Electronic City Phase II', 'total_sqft': 1056.0, 'bath': 2, 'balcony': 1, 'bedrooms': 2}])

In [47]:
model.predict(example)

array([34.79409895])

In [48]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

In [49]:
df.to_csv('Cleaned_data.csv')